In [15]:
import matplotlib.pyplot as plt
import os
import skimage
from skimage.io import imread,imsave
from skimage.filters import threshold_otsu
import numpy as np


In [16]:
def whole_dice_metric(y_pred,y_true):
    smooth = 10e-16
    # single image so just roll it out into a 1D array
    
    m1 =np.reshape(y_pred,(-1))/255
    m2 =np.reshape(y_true,(-1))/255
    
    
    intersection = (m1 * m2)

    score = 2. * (np.sum(intersection) + smooth) / (np.sum(m1) +(np.sum(m2) + smooth))
        
    return score

In [24]:

img_list=os.listdir('prediction')
avg=0
for img_name in img_list:
    img_pred=imread('prediction/'+img_name)
    gt=imread('processed_data/nucleus_maps/'+img_name.split('.')[0].split('_')[1]+'_nucleus_mask.png')
    thresh=threshold_otsu(img_pred)
    img_pred=img_pred>thresh
    dice=whole_dice_metric(img_pred.astype(np.float32)*255,gt)
    avg+=dice
    print(img_name.split('.')[0].split('_')[1],' : ',dice)
print(avg/len(img_list))

TCGA-E2-A14V-01Z-00-DX1  :  0.8333296281750119
TCGA-21-5786-01Z-00-DX1  :  0.7201361766716904
TCGA-21-5784-01Z-00-DX1  :  0.7638048712223855
TCGA-B0-5710-01Z-00-DX1  :  0.7940773464459038
TCGA-G2-A2EK-01A-02-TSB  :  0.762075211316198
TCGA-CH-5767-01Z-00-DX1  :  0.7817884242122582
TCGA-RD-A8N9-01A-01-TS1  :  0.8248413245090023
TCGA-G9-6348-01Z-00-DX1  :  0.7923000165947918
TCGA-DK-A2I6-01A-01-TS1  :  0.8673273565170285
TCGA-KB-A93J-01A-01-TS1  :  0.8257282747761047
TCGA-AY-A8YK-01A-01-TS1  :  0.7633746834126345
TCGA-E2-A1B5-01Z-00-DX1  :  0.7655227486515934
TCGA-NH-A8F7-01A-01-TS1  :  0.7900778338393527
TCGA-HE-7128-01Z-00-DX1  :  0.7047852115651219
0.7849406505649341
